In [1]:
import csv
import firebase_admin
import google.cloud
from firebase_admin import credentials, firestore

In [2]:
if not firebase_admin._apps:
    cred = credentials.Certificate("./firestore-key.json")
    app = firebase_admin.initialize_app(cred)

In [15]:
store = firestore.client()
file_path = "foodbanks.csv"
collection_name = "foodbanks_LA"

In [16]:
def batch_data(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [17]:
data = []
headers = []
with open(file_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            for header in row:
                headers.append(header)
            line_count += 1
        else:
            obj = {}
            for idx, item in enumerate(row):
                obj[headers[idx]] = item
            data.append(obj)
            line_count += 1
    print(f'Processed {line_count} lines.')

Processed 45 lines.


In [18]:
for batched_data in batch_data(data, 499):
    batch = store.batch()
    for data_item in batched_data:
        doc_ref = store.collection(collection_name).document()
        batch.set(doc_ref, data_item)
    batch.commit()

print('Done')

Done
